# Section 508.3.3 - Separation
## Building Code
```
No separation is required between nonseparated occupancies. 

Exceptions: 

 1. Group H-2, H-3, H-4 and H-5 occupancies shall be separated from all other occupancies in accordance with Section 508.4. 

 2. Group I-1, R-1, R-2 and R-3 dwelling units and sleeping units  shall be separated from other dwelling or sleeping units and from other occupancies contiguous to them in accordance with the requirements of Section 420.
```



In [1]:
import rdflib
import pandas as pd
# Load RDF graph
rdf_graph = rdflib.Graph()
rdf_graph.parse("..\..\..\LBD files\smartReview\outputHotel Compliant Revit 2023 Advanced with Spaces.ifc.ttl", format="ttl")

<Graph identifier=N6536d51c7ffe4517a3bc59c5bfa90528 (<class 'rdflib.graph.Graph'>)>

In [19]:
from IPython.display import display, HTML
with open("..\..\..\SPARQL\\uniqueOccupancy.sparql", "r") as file:
    sparql_query = file.read()
    
results = rdf_graph.query(sparql_query)

# Process and format the results
story_dict = {}

for row in results:
    level_value = row[0]
    occupancies = row[1].split(", ")
    
    if level_value not in story_dict:
        story_dict[level_value] = []
    
    story_dict[level_value].extend(occupancies)

data = []
# Print the results in the desired format
for level, occupancies in sorted(story_dict.items()):
    unique_occupancies = sorted(set(occupancies))  # Remove duplicates and sort for better display
    occupancy_count = len(unique_occupancies)
    
    contains_exception1 = any(o in ["H-2", "H-3", "H-4", "H-5"] for o in unique_occupancies)
    contains_exception2 = any(o in ["I-1", "R-1", "R-2", "R-3"] for o in unique_occupancies)
    
    if occupancy_count == 1:
        exception1_result = "Passed"
        exception2_result = "Passed"
        reason = "The Separation Group only contains one occupancy. There can be no violation of Exception 1 or Exception 2."
    else:
        exception1_result = "Passed" if contains_exception1 else "Failed"
        exception2_result = "Passed" if contains_exception2 else "Failed"
        
        reason_parts = []
        if not contains_exception1:
            reason_parts.append("The Separation Group does not contain an occupancy described in Exception 1.")
        if not contains_exception2:
            reason_parts.append("The Separation Group does not contain an occupancy described in Exception 2.")
        
        if reason_parts:
            reason = " ".join(reason_parts)
        else:
            reason = "The Separation Group does not contain the required occupancies for Exception 1 or Exception 2."
        
    data.append({
        "Separation Group": level,
        "Occupancies": ", ".join(occupancies),
        "Exception 1 Result": exception1_result,
        "Exception 2 Result": exception2_result,
        "Reason": reason
    })

# Create DataFrame
df = pd.DataFrame(data)




In [20]:
# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame as a scrollable table
def render_scrollable_table(dataframe, max_height=300):
    display(HTML(f"""
    <div style="max-height: {max_height}px; overflow-y: scroll;">
        {dataframe.to_html(index=False)}
    </div>
    """))

render_scrollable_table(df)

Separation Group,Occupancies,Exception 1 Result,Exception 2 Result,Reason
01,"A-3, A-2, R-1, B",Failed,Passed,The Separation Group does not contain an occupancy described in Exception 1.
02,R-1,Passed,Passed,The Separation Group only contains one occupancy. There can be no violation of Exception 1 or Exception 2.
03,R-1,Passed,Passed,The Separation Group only contains one occupancy. There can be no violation of Exception 1 or Exception 2.
04,R-1,Passed,Passed,The Separation Group only contains one occupancy. There can be no violation of Exception 1 or Exception 2.
